In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle


##LOADING DATA

In [ ]:
movies= pd.read_csv("tmdb_5000_movies.csv")
credits= pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [ ]:
movies.shape

(4803, 20)

In [ ]:
credits.shape

(4803, 4)

In [ ]:
data = movies.merge(credits,on='title')
data

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]",...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,9367,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,72766,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"":

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [ ]:
data.shape

(4809, 23)

In [ ]:
data.isnull().sum()

budget                     0
genres                     0
homepage                3096
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
movie_id                   0
cast                       0
crew                       0
dtype: int64

#FEATURE SELECTION

In [ ]:
features_selected=["genres","movie_id","keywords","title","cast","crew","overview"]
data = data[features_selected]

In [ ]:
data

,genres,movie_id,keywords,title,cast,crew,overview
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Following the death of District Attorney Harve...
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...
4804,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",El Mariachi just wants to play his guitar and ...
4805,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",72766,[],Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",A newlywed couple's honeymoon is upended by th...
4806,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","""Signed, Sealed, Delivered"" introduces a dedic..."
4807,[],126186,[],Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",When ambitious New York attorney Sam is sent t...


In [ ]:
data[features_selected].isnull().sum()

genres      0
movie_id    0
keywords    0
title       0
cast        0
crew        0
overview    3
dtype: int64

In [ ]:
data.dropna(inplace=True)

<ipython-input-65-f1116dacf2bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [ ]:
data.isnull().sum()

genres      0
movie_id    0
keywords    0
title       0
cast        0
crew        0
overview    0
dtype: int64

In [ ]:
data.duplicated().sum()

0

In [ ]:
data['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [ ]:
import ast

def convert(obj):
  lst = []
  for i in ast.literal_eval(obj):
    lst.append(i['name'])
  return lst

data['genres']=data['genres'].apply(convert)


<ipython-input-69-c8100e3a4831>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres']=data['genres'].apply(convert)


In [ ]:
data

,genres,movie_id,keywords,title,cast,crew,overview
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di..."
1,"[Adventure, Fantasy, Action]",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha..."
2,"[Action, Adventure, Crime]",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...
3,"[Action, Crime, Drama, Thriller]",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Following the death of District Attorney Harve...
4,"[Action, Adventure, Science Fiction]",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]",9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",El Mariachi just wants to play his guitar and ...
4805,"[Comedy, Romance]",72766,[],Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",A newlywed couple's honeymoon is upended by th...
4806,"[Comedy, Drama, Romance, TV Movie]",231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","""Signed, Sealed, Delivered"" introduces a dedic..."
4807,[],126186,[],Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",When ambitious New York attorney Sam is sent t...


In [ ]:
data['keywords']=data['keywords'].apply(convert)

<ipython-input-71-82c8b7128114>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['keywords']=data['keywords'].apply(convert)


In [ ]:
data

,genres,movie_id,keywords,title,cast,crew,overview
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha..."
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Following the death of District Attorney Harve...
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]",9367,"[united states–mexico barrier, legs, arms, pap...",El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",El Mariachi just wants to play his guitar and ...
4805,"[Comedy, Romance]",72766,[],Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",A newlywed couple's honeymoon is upended by th...
4806,"[Comedy, Drama, Romance, TV Movie]",231617,"[date, love at first sight, narration, investi...","Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","""Signed, Sealed, Delivered"" introduces a dedic..."
4807,[],126186,[],Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",When ambitious New York attorney Sam is sent t...


In [ ]:
def convertcast(obj):
  lst=[]
  cntr=0
  for i in ast.literal_eval(obj):
    if cntr==3:
      break
    cntr+=1
    lst.append(i['name'])
  return lst


In [ ]:
data['cast'] = data['cast'].apply(convertcast)

<ipython-input-74-c11fd741b10d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cast'] = data['cast'].apply(convertcast)


In [ ]:
data

,genres,movie_id,keywords,title,cast,crew,overview
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha..."
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Following the death of District Attorney Harve...
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]",9367,"[united states–mexico barrier, legs, arms, pap...",El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",El Mariachi just wants to play his guitar and ...
4805,"[Comedy, Romance]",72766,[],Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein]","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",A newlywed couple's honeymoon is upended by th...
4806,"[Comedy, Drama, Romance, TV Movie]",231617,"[date, love at first sight, narration, investi...","Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe]","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","""Signed, Sealed, Delivered"" introduces a dedic..."
4807,[],126186,[],Shanghai Calling,"[Daniel Henney, Eliza Coupe, Bill Paxton]","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",When ambitious New York attorney Sam is sent t...


In [ ]:
data['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [ ]:
def convertcrew(obj):
  lst=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      lst.append(i['name'])
      break
  return lst

In [ ]:
data['crew'] = data['crew'].apply(convertcrew)

<ipython-input-78-2b3e550f5681>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['crew'] = data['crew'].apply(convertcrew)


In [ ]:
data

,genres,movie_id,keywords,title,cast,crew,overview
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"In the 22nd century, a paraplegic Marine is di..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"Captain Barbossa, long believed to be dead, ha..."
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],A cryptic message from Bond’s past sends him o...
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],Following the death of District Attorney Harve...
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],"John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]",9367,"[united states–mexico barrier, legs, arms, pap...",El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez],El Mariachi just wants to play his guitar and ...
4805,"[Comedy, Romance]",72766,[],Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein]",[Edward Burns],A newlywed couple's honeymoon is upended by th...
4806,"[Comedy, Drama, Romance, TV Movie]",231617,"[date, love at first sight, narration, investi...","Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe]",[Scott Smith],"""Signed, Sealed, Delivered"" introduces a dedic..."
4807,[],126186,[],Shanghai Calling,"[Daniel Henney, Eliza Coupe, Bill Paxton]",[Daniel Hsia],When ambitious New York attorney Sam is sent t...


In [ ]:
data['crew'][0]

['James Cameron']

In [ ]:
type(data['overview'][0])

str

In [ ]:
data['overview'] = data['overview'].apply(lambda x:x.split())

<ipython-input-82-6567db2fdda5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['overview'] = data['overview'].apply(lambda x:x.split())


In [ ]:
data

,genres,movie_id,keywords,title,cast,crew,overview
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],"[Following, the, death, of, District, Attorney..."
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]",9367,"[united states–mexico barrier, legs, arms, pap...",El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez],"[El, Mariachi, just, wants, to, play, his, gui..."
4805,"[Comedy, Romance]",72766,[],Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein]",[Edward Burns],"[A, newlywed, couple's, honeymoon, is, upended..."
4806,"[Comedy, Drama, Romance, TV Movie]",231617,"[date, love at first sight, narration, investi...","Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe]",[Scott Smith],"[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,[],126186,[],Shanghai Calling,"[Daniel Henney, Eliza Coupe, Bill Paxton]",[Daniel Hsia],"[When, ambitious, New, York, attorney, Sam, is..."


In [ ]:
data['keywords'] = data['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
data['cast'] = data['cast'].apply(lambda x: [i.replace(" ","") for i in x])
data['genres'] = data['genres'].apply(lambda x: [i.replace(" ","") for i in x])
data['crew'] = data['crew'].apply(lambda x: [i.replace(" ","") for i in x])

<ipython-input-84-ebb10916926e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['keywords'] = data['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
<ipython-input-84-ebb10916926e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cast'] = data['cast'].apply(lambda x: [i.replace(" ","") for i in x])
<ipython-input-84-ebb10916926e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [ ]:
data

,genres,movie_id,keywords,title,cast,crew,overview
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Following, the, death, of, District, Attorney..."
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...",John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]",9367,"[unitedstates–mexicobarrier, legs, arms, paper...",El Mariachi,"[CarlosGallardo, JaimedeHoyos, PeterMarquardt]",[RobertRodriguez],"[El, Mariachi, just, wants, to, play, his, gui..."
4805,"[Comedy, Romance]",72766,[],Newlyweds,"[EdwardBurns, KerryBishé, MarshaDietlein]",[EdwardBurns],"[A, newlywed, couple's, honeymoon, is, upended..."
4806,"[Comedy, Drama, Romance, TVMovie]",231617,"[date, loveatfirstsight, narration, investigat...","Signed, Sealed, Delivered","[EricMabius, KristinBooth, CrystalLowe]",[ScottSmith],"[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,[],126186,[],Shanghai Calling,"[DanielHenney, ElizaCoupe, BillPaxton]",[DanielHsia],"[When, ambitious, New, York, attorney, Sam, is..."


In [ ]:
data['tags'] = data['overview'] + data['genres'] + data['keywords'] + data['cast'] + data['crew']

<ipython-input-86-946d1b3e4e47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tags'] = data['overview'] + data['genres'] + data['keywords'] + data['cast'] + data['crew']


In [ ]:
data

,genres,movie_id,keywords,title,cast,crew,overview,tags
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin...","[In, the, 22nd, century,, a, paraplegic, Marin..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d...","[Captain, Barbossa,, long, believed, to, be, d..."
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send...","[A, cryptic, message, from, Bond’s, past, send..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Following, the, death, of, District, Attorney...","[Following, the, death, of, District, Attorney..."
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...",John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili...","[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]",9367,"[unitedstates–mexicobarrier, legs, arms, paper...",El Mariachi,"[CarlosGallardo, JaimedeHoyos, PeterMarquardt]",[RobertRodriguez],"[El, Mariachi, just, wants, to, play, his, gui...","[El, Mariachi, just, wants, to, play, his, gui..."
4805,"[Comedy, Romance]",72766,[],Newlyweds,"[EdwardBurns, KerryBishé, MarshaDietlein]",[EdwardBurns],"[A, newlywed, couple's, honeymoon, is, upended...","[A, newlywed, couple's, honeymoon, is, upended..."
4806,"[Comedy, Drama, Romance, TVMovie]",231617,"[date, loveatfirstsight, narration, investigat...","Signed, Sealed, Delivered","[EricMabius, KristinBooth, CrystalLowe]",[ScottSmith],"[""Signed,, Sealed,, Delivered"", introduces, a,...","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,[],126186,[],Shanghai Calling,"[DanielHenney, ElizaCoupe, BillPaxton]",[DanielHsia],"[When, ambitious, New, York, attorney, Sam, is...","[When, ambitious, New, York, attorney, Sam, is..."


In [ ]:
new_data = data[['movie_id','title','tags']]

In [ ]:
new_data

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [ ]:
new_data['tags'] = new_data['tags'].apply(lambda x:" ".join(x))

<ipython-input-90-7f6e76707cd0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tags'] = new_data['tags'].apply(lambda x:" ".join(x))


In [ ]:
new_data

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...


In [ ]:
new_data['tags'] = new_data['tags'].apply(lambda x:x.lower())

<ipython-input-92-c5247928e45b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tags'] = new_data['tags'].apply(lambda x:x.lower())


In [ ]:
new_data

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [ ]:
new_data.to_csv("movies.csv",index=False)

# for use , start from here


In [1]:
new_data = pd.read_csv("movies.csv")

<IPython.core.display.Javascript object>

In [2]:
new_data

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4801,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4802,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4803,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4804,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [3]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [4]:
def stemming(text):
  lst=[]
  for i in text.split():
    lst.append(ps.stem(i))

  return " ".join(lst)

In [5]:
new_data['tags'] = new_data['tags'].apply(stemming)

# Data vectorization

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [7]:
vectors = cv.fit_transform(new_data['tags']).toarray()

In [8]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [9]:

cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

# Distance measure

In [12]:
sim_matrix = cosine_similarity(vectors)

In [13]:
sim_matrix[0][0]

1.0000000000000002

In [ ]:
def recommender(name):
  idx = new_data[new_data['title']==name].index[0]
  distance = sim_matrix[idx]
  movie_lst = sorted(list(enumerate(distance)),reverse=True,key = lambda x:x[1])[1:6]

  for i in movie_lst:
    print(new_data.iloc[i[0]].title)



In [ ]:
try:
  recommender('Batman')
except:
  print("No suggetions found")


Batman
Batman & Robin
Batman Begins
Batman Returns
The R.M.


In [ ]:
pickle.dump(new_data,open("movies_list.pkl","wb"))

In [ ]:
pickle.dump(sim_matrix,open("sim_matrix.pkl","wb"))

In [ ]:
def fetch_poster(movie_name):
    movie_index = new_data[new_data['title']==movie_name].index[0]
    movie_id = new_data.iloc[movie_index].movie_id
    print(movie_id)

In [ ]:
fetch_poster('Batman')

268


In [ ]:
new_data[new_data['title']=='Batman']

,movie_id,title,tags
1362,268,Batman,the dark knight of gotham citi begin hi war on...
1363,2661,Batman,the dark knight of gotham citi begin hi war on...
1364,268,Batman,the dynam duo face four super-villain who plan...
1365,2661,Batman,the dynam duo face four super-villain who plan...


In [14]:
top_indices = np.argsort(-sim_matrix, axis=1)[:, :6]  # Sort in descending order

# Get the top 6 values of each row using the indices
top_values = np.take_along_axis(sim_matrix, top_indices, axis=1)

# Print the top 6 values and indices for the first few rows as an example
for i in range(5):  # Print top 6 for the first 5 rows
    print(f"Row {i+1}:")
    print("Top Values:", top_values[i])
    print("Top Indices:", top_indices[i])
    print()

Row 1:
Top Values: [1.         0.28676967 0.26901379 0.26051302 0.25560859 0.2503867 ]
Top Indices: [   0 1216 2409 3730  507  539]

Row 2:
Top Values: [1.         0.41400434 0.27500955 0.2533202  0.2057983  0.2057983 ]
Top Indices: [   1   12  199   17  216 3572]

Row 3:
Top Values: [1.         0.36099744 0.33071891 0.3263956  0.26111648 0.23904572]
Top Indices: [   2   11   29 1346 4077 1749]

Row 4:
Top Values: [1.         0.42256142 0.32470271 0.32025631 0.31815821 0.31774445]
Top Indices: [   3   65  428 1363  299  119]

Row 5:
Top Values: [1.         0.29428682 0.29238044 0.28198718 0.27397779 0.27288841]
Top Indices: [   4 1322 3093 3377  610 1257]



In [23]:
idx = np.argmax(sim_matrix[0][1:])

In [24]:
idx

1215

In [22]:
sim_matrix

(4806, 4806)

In [25]:
top_values.shape

(4806, 6)

In [26]:
pickle.dump(top_values,open("simi_matrix.pkl","wb"))

In [65]:

similarity = pickle.load(open('sim_matrix.pkl','rb'))
simi = pickle.load(open('simi_matrix.pkl','rb'))

In [28]:
print(similarity[0][:7])

[1.         0.08346223 0.0860309  0.07347184 0.18929941 0.10838875
 0.04024218]


In [29]:
print(simi[0][:7])

[1.         0.28676967 0.26901379 0.26051302 0.25560859 0.2503867 ]


In [30]:
distances = sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])

In [32]:
print(distances[:7])

[(0, 1.0000000000000002), (1216, 0.28676966733820225), (2409, 0.26901379342448517), (3730, 0.2605130246476754), (507, 0.255608593705383), (539, 0.25038669783359574), (582, 0.24511108480187255)]


In [52]:
simil = []
for i in range(len(similarity)):
    distances=[]
    distances = sorted(list(enumerate(similarity[i])), reverse=True, key=lambda x: x[1])[1:6]
    simil.append(distances)


In [64]:
pickle.dump(simil,open("simi_matrix.pkl","wb"))

In [66]:
simi

[[(1216, 0.28676966733820225),
  (2409, 0.26901379342448517),
  (3730, 0.2605130246476754),
  (507, 0.255608593705383),
  (539, 0.25038669783359574)],
 [(12, 0.4140043409440133),
  (199, 0.27500954910846337),
  (17, 0.2533201985524494),
  (216, 0.20579830217101058),
  (3572, 0.20579830217101058)],
 [(11, 0.3609974361905776),
  (29, 0.3307189138830738),
  (1346, 0.32639560491693337),
  (4077, 0.2611164839335467),
  (1749, 0.2390457218668787)],
 [(65, 0.42256142372325745),
  (428, 0.3247027084163318),
  (1363, 0.3202563076101742),
  (299, 0.3181582090733562),
  (119, 0.3177444546511212)],
 [(1322, 0.2942868240199161),
  (3093, 0.29238044084454384),
  (3377, 0.2819871785071246),
  (610, 0.27397779068244577),
  (1257, 0.2728884114549076)],
 [(30, 0.5139561687500468),
  (159, 0.4717281765248634),
  (38, 0.3340765523905306),
  (20, 0.3107594384269423),
  (1540, 0.2631984023788486)],
 [(2315, 0.20640627484613444),
  (269, 0.20380986614602725),
  (255, 0.19138975058773822),
  (1682, 0.18605210